In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path
from collections import OrderedDict

In [2]:
wikicssh_path = Path("../data/v1")
wikicssh_files = list(wikicssh_path.glob("./*.csv"))
wikicssh_files

[WindowsPath('../data/v1/WikiCSSH_categories.csv'),
 WindowsPath('../data/v1/WikiCSSH_category2page.csv'),
 WindowsPath('../data/v1/WikiCSSH_category_links.csv'),
 WindowsPath('../data/v1/WikiCSSH_category_links_all.csv'),
 WindowsPath('../data/v1/Wikicssh_core_categories.csv'),
 WindowsPath('../data/v1/WikiCSSH_page2redirect.csv')]

In [3]:
df_categories = pd.read_csv('../data/v1/WikiCSSH_categories.csv')
df_categories.head()

,category,level
0,Computer_science,1
1,Mathematics,1
2,Information_science,1
3,Computer_engineering,1
4,Statistics,1


In [4]:
df_categories.shape

(7354, 2)

In [5]:
("Category:"+ df_categories.category).head()

0        Category:Computer_science
1             Category:Mathematics
2     Category:Information_science
3    Category:Computer_engineering
4              Category:Statistics
Name: category, dtype: object

In [6]:
df_categories.set_index("category").head().to_dict(orient="index", into=OrderedDict)

OrderedDict([('Computer_science', {'level': 1}),
             ('Mathematics', {'level': 1}),
             ('Information_science', {'level': 1}),
             ('Computer_engineering', {'level': 1}),
             ('Statistics', {'level': 1})])

In [7]:
df_category_links = pd.read_csv('../data/v1/WikiCSSH_category_links.csv')
df_category_links.head()

,parent_cat,child_cat,parent_level,child_level
0,Computer_engineering,Electronic_engineering,1,2
1,Computer_engineering,Software_engineering,1,2
2,Computer_engineering,Computer_hardware,1,2
3,Computer_engineering,Computer_systems,1,2
4,Computer_engineering,Computer_engineering_stubs,1,2


In [8]:
df_category_links.shape

(8323, 4)

In [9]:
len(set(df_categories.category.values))

7354

In [10]:
len(set(df_category_links.parent_cat.values) | set(df_category_links.child_cat.values))

7263

In [11]:
df_pages = pd.read_csv('../data/v1/WikiCSSH_category2page.csv')
df_pages.head()

,cat_title,cat_level,num_aff_pages,page_title
0,Information_science,1,247,Information_professional
1,Computer_science,1,50,Technology_transfer_in_computer_science
2,Computer_science,1,50,Computational_social_choice
3,Computer_science,1,50,Outline_of_computer_science
4,Computer_science,1,50,Reduction_Operator


In [12]:
df_pages.shape

(272747, 4)

In [13]:
df_pages.page_title.value_counts().shape

(181070,)

In [14]:
df_redirects = pd.read_csv('../data/v1/WikiCSSH_page2redirect.csv')
df_redirects.head()

,page_title,min_level,redirect_page_title
0,Information_professional,1,Information_Professional
1,Technology_transfer_in_computer_science,1,NaN
2,Computational_social_choice,1,Computational_social_choice
3,Outline_of_computer_science,1,Computer_science_basic_topics
4,Outline_of_computer_science,1,List_of_computer_science_topics


In [15]:
df_redirects.redirect_page_title.value_counts().shape

(580312,)

In [16]:
df_redirects.shape

(942295, 3)

In [17]:
isolate_cats = (
    set(df_categories.category.values) 
    - set(
        set(df_category_links.parent_cat.values) 
        | set(df_category_links.child_cat.values)
    )
)
len(isolate_cats)

91

In [18]:
root_child_cats = set(
    set(df_category_links.parent_cat.values) 
    - set(df_category_links.child_cat.values)
)
len(root_child_cats)

76

In [19]:
list(root_child_cats)[:10]

['Asbestos',
 'Naming_conventions',
 'Steampunk',
 'Trains',
 'French-language_media_in_Canada',
 'Crops',
 'Firearms',
 'Information_science',
 'Aviation_standards',
 'Air_navigation']

In [20]:
df_categories[df_categories.category.isin(root_child_cats)]

,category,level
0,Computer_science,1
1,Mathematics,1
2,Information_science,1
3,Computer_engineering,1
4,Statistics,1
...,...,...
6610,Plant_disease_scales,17
6629,Sustainable_design,17
6876,Masks,18
7118,Economic_reforms,19


In [21]:
df_categories[df_categories.category.isin(isolate_cats)].head()

,category,level
2390,Microphones,5
3166,Kernel_programmers,5
4332,Silicon_photonics,7
4404,Silicones,7
4578,ITU-T_recommendations,7


In [22]:
df_categories

,category,level
0,Computer_science,1
1,Mathematics,1
2,Information_science,1
3,Computer_engineering,1
4,Statistics,1
...,...,...
7349,Techniques,20
7350,Streptococcal_proteins,20
7351,Catalysts,20
7352,Flax,20


In [23]:
df_category_links_all = pd.concat([
    df_category_links,
    df_categories[df_categories.category.isin(root_child_cats)].rename(columns={
        "category": "child_cat", 
        "level": "child_level", #	parent_level	child_level
    }).assign(parent_cat="<ROOT>", parent_level=0),
    pd.DataFrame({
        "parent_cat": [""],
        "child_cat": ["<ROOT>"],
        "parent_level": [-1],
        "child_level": [0],
    })
#     df_categories[df_categories.category.isin(isolate_cats)].rename(columns={
#         "category": "child_cat", 
#         "level": "child_level", #	parent_level	child_level
#     }).assign(parent_cat="", parent_level=0)
    
], axis=0, sort=True).sort_values(["parent_level", "child_level"])
df_category_links_all.shape

(8400, 4)

In [24]:
df_category_links_all.head()

,child_cat,child_level,parent_cat,parent_level
0,<ROOT>,0,,-1
0,Computer_science,1,<ROOT>,0
1,Mathematics,1,<ROOT>,0
2,Information_science,1,<ROOT>,0
3,Computer_engineering,1,<ROOT>,0


In [25]:
df_category_links_all[
    #(df_category_links_all.parent_cat=="Artificial_intelligence")
    (df_category_links_all.child_cat=="Artificial_intelligence")
]

,child_cat,child_level,parent_cat,parent_level
85,Artificial_intelligence,3,Areas_of_computer_science,2
492,Artificial_intelligence,3,Unsolved_problems_in_computer_science,2


In [26]:
df_categories[df_categories.level<=5].shape

(3389, 2)

In [27]:
G = nx.DiGraph()

In [28]:
G.add_edges_from(df_category_links_all.set_index(["parent_cat", "child_cat"]).to_dict(orient="index", into=OrderedDict))
G.size()

8400

In [29]:
nx.algorithms.is_directed_acyclic_graph(G)

True

In [30]:
node = "Artificial_intelligence"
list(G.neighbors(node))

['Fuzzy_logic',
 'Machine_learning',
 'Turing_tests',
 'Rule_engines',
 'Robots',
 'Computer_vision',
 'Logic_programming',
 'Open_source_artificial_intelligence',
 'Virtual_assistants',
 'Cloud_robotics',
 'Game_artificial_intelligence',
 'AI_accelerators',
 'Affective_computing',
 'Cognitive_architecture',
 'Artificial_intelligence_applications',
 'Artificial_immune_systems',
 'Robotics',
 'Artificial_Intelligence_existential_risk',
 'Evolutionary_computation']

In [31]:
list(G.predecessors(node))

['Areas_of_computer_science', 'Unsolved_problems_in_computer_science']

In [32]:
%%time
cat2pages = pd.read_csv('../data/v1/WikiCSSH_category2page.csv').groupby("cat_title").page_title.agg(list)

Wall time: 1.2 s


In [33]:
from IPython.display import display, HTML

In [34]:
def show_context_of_category(node):
    parents = list(G.predecessors(node))
    children = list(G.neighbors(node))
    table_header = "<thead><tr><th>Parents</th><th>Node</th><th>Children</th></tr></thead>"
    parents_list = "\n".join([f"<li><a href='https://en.wikipedia.org/wiki/Category:{i}'>{i}</a></li>" for i in parents])
    children_list = "\n".join([f"<li><a href='https://en.wikipedia.org/wiki/Category:{i}'>{i}</a></li>" for i in children])
    node_list = f"<li><a href='https://en.wikipedia.org/wiki/Category:{node}'>{node}</a></li>"
    page_list = " | ".join([
        f"<a href='https://en.wikipedia.org/wiki/{i}'>{i}</a>"
        for i in cat2pages[node]
    ])
    table_body = f"""<tbody>
    <tr>
        <td><ul>{parents_list}</ul></td>
        <td style='background-color: pink'>{node_list}</td>
        <td><ul>{children_list}</ul></td>
    </tr>
    <tr><td colspan='3'><strong>Pages</strong><br/>{page_list}</td></tr>
    </tbody>"""
    div = f"""
    <style>.wikicssh td, .wikicssh th {{text-align:left}}</style>
    <div class='wikicssh'>
    <table>{table_header}{table_body}</table>
    </div>"""
    return HTML(div)

In [35]:
show_context_of_category(node)

Parents,Node,Children
Areas_of_computer_science Unsolved_problems_in_computer_science,Artificial_intelligence,Fuzzy_logic Machine_learning Turing_tests Rule_engines Robots Computer_vision Logic_programming Open_source_artificial_intelligence Virtual_assistants Cloud_robotics Game_artificial_intelligence AI_accelerators Affective_computing Cognitive_architecture Artificial_intelligence_applications Artificial_immune_systems Robotics Artificial_Intelligence_existential_risk Evolutionary_computation


## Interactive usage

In [36]:
from ipywidgets import interact_manual

In [37]:
interact_manual(show_context_of_category, node="Artificial_intelligence");

interactive(children=(Text(value='Artificial_intelligence', description='node'), Button(description='Run Inter…